In [1]:
"""
data load

"""

import cv2
import os
import numpy as np
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
import natsort


def load_data(PATH, split=0.21):
    images = natsort.natsorted(glob(os.path.join(PATH, "/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resized_images_single/*")))
    masks = natsort.natsorted(glob(os.path.join(PATH, "/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resized_masks_single/*")))

    print(images)
    print(masks)

    total_size = len(images)
    total_mask_size = len(masks)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    print("total_number_of_images:", total_size)
    
    print("total_mask_images:", total_mask_size)
    print("valid_size:", valid_size)
    print("test_size:", test_size)


    train_x, valid_x = train_test_split(
             images,
             test_size=valid_size,
             random_state=42
     )
    train_y, valid_y = train_test_split(
             masks,
             test_size=valid_size,
             random_state=42
     )

    train_x, test_x = train_test_split(
             images,
             test_size=test_size,
             random_state=42
     )
    train_y, test_y = train_test_split(
             masks,
             test_size=test_size,
             random_state=42
     )

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


def read_image(PATH):
     PATH = PATH.decode()
     x = cv2.imread(PATH, cv2.IMREAD_COLOR)
     x = cv2.resize(x, (128, 128))
     x = x/255.0
     return x


def read_mask(PATH):
     PATH = PATH.decode()
     x = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)
     x = cv2.resize(x, (128, 128))
     x = x/255.0
     x = np.expand_dims(x, axis=-1)
     return x


def tf_parse(sx, sy):
     def _parse(x, y):
         x = read_image(x)
         y = read_mask(y)
         return x, y

     sx, sy = tf.numpy_function(_parse, [sx, sy], [tf.float64, tf.float64])
     sx.set_shape([128, 128, 3])
     sy.set_shape([128, 128, 1])
     return sx, sy


def tf_dataset(x, y, batch=8):
     dataset = tf.data.Dataset.from_tensor_slices((x, y))
     dataset = dataset.map(tf_parse)
     dataset = dataset.batch(batch)
     dataset = dataset.repeat()
     return dataset


if __name__ == "__main__":
     PATH = "/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/datasets/"
     (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(PATH)

     print("Training data x: ", len(train_x))
     print("Training data y: ", len(train_y))
     print("Validation data x: ", len(valid_x))
     print("Validation data y: ", len(valid_y))
     print("Testing data x: ", len(test_x))
     print("Testing data y: ", len(test_y))
     print(" ")

     ds = tf_dataset(test_x, test_y)
     for x, y in ds:
         print(x.shape, y.shape)
         print('test_x:', test_x)
         print('test_y:', test_y)
         print('-----')
         print('train_x:', train_x)
         print('train_y:', train_y)
         print('-----')
         print('valid_x:', valid_x)
         print('valid_y:', valid_y)
         print('-----')
         break
'''
# put all the content in a dict
tostore=dict(zip(['X_train', 'X_test', 'y_train', 'y_test' ], [train, test, y_train, y_test))
# open a file with context manager
with open('REP'+str(n),'wb') as filestore: 
    pkl.dump(tostore,filestore) # write dict to file
    #pkl.load() #load back the data

joblib.dump(train_test_split, "/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/datasets/")
#my_train_test_split = joblib.load(path)

'''

['/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resized_images_single/scene_0.jpg', '/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resized_images_single/scene_1.jpg', '/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resized_images_single/scene_2.jpg', '/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resized_images_single/scene_3.jpg', '/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resized_images_single/scene_4.jpg', '/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resized_images_single/scene_5.jpg', '/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resized_images_single/scene_6.jpg', '/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resized_images_single/scene_7.jpg', '/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resized_images_single/scene_8.jpg', '/content/drive/MyDrive/Colab Notebooks/My work/billboard_datasets/resiz

'\n# put all the content in a dict\ntostore=dict(zip([\'X_train\', \'X_test\', \'y_train\', \'y_test\' ], [train, test, y_train, y_test))\n# open a file with context manager\nwith open(\'REP\'+str(n),\'wb\') as filestore: \n    pkl.dump(tostore,filestore) # write dict to file\n    #pkl.load() #load back the data\n\njoblib.dump(train_test_split, "/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/datasets/")\n#my_train_test_split = joblib.load(path)\n\n'

In [ ]:
from keras.utils import CustomObjectScope

BATCH = 8

test_dataset = tf_dataset(test_x, test_y, batch=BATCH)
test_steps = len(test_x) // BATCH


if len(test_x) % BATCH != 0:
    test_steps += 1

#model.evaluate(test_dataset, steps=test_steps)
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

with CustomObjectScope({'iou': iou}):

     unetModel = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/model/billboard_unet_seg.hdf5")
    
    #unetModel = load_model("/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/model/unet_model.h5")

unetModel.evaluate(test_dataset, steps=test_steps)

102/102 [==============================] - 1283s 13s/step - loss: 0.0417 - acc: 0.9794 - recall_1: 0.8782 - precision_1: 0.9680 - iou: 0.7907


[0.0416654609143734,
 0.9793557524681091,
 0.8782221674919128,
 0.9680233597755432,
 0.7906643152236938]

In [6]:
from tqdm import tqdm
from keras.utils import CustomObjectScope
import cv2
import matplotlib.pyplot as plt



def read_image(PATH):
    x = cv2.imread(PATH, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (128, 128))
    #print("image shape:",x.shape)#(128, 128, 3)
    x = x/255.0
    return x


def read_mask(PATH):
    x = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (128, 128))
    #print("mask:",x.shape)#(128, 128)
    #x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

with CustomObjectScope({'iou': iou}):
      
      unetModel = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/model/billboard_unet_seg.hdf5")
      #unetModel = load_model("/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/model/unet_model.h5")

for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        x = read_image(x)
        y = read_mask(y)
      
        print('test x:', test_x[i])
        textx_file_name = os.path.basename(test_x[i])
        print('test x:', textx_file_name)

        
        y_pred = unetModel.predict(np.expand_dims(x, axis=0))
        y_predprob = np.squeeze(y_pred) * 255.0

        y_pred = y_pred[0] > 0.5
        y_pred = mask_parse(y_pred) * 255.0
        print('y_predprobshape:', y_predprob.shape)#(128, 128)
        print('y_pred shape:', y_pred.shape)#(128, 128, 3)
 
        #y_pred = (model.predict(np.expand_dims(x, axis=0))>= 0.5).astype(int)
        #print('y_pred shape:', y_pred.shape)#(1, 128, 128, 1)
        #y_pred = y_pred.reshape(y_pred.shape[1:])
        #print('y_pred reshape:', y_pred.shape)#(128, 128, 1)
        
        '''
        plt.subplot(131)
        plt.imshow(x)
        plt.axis('off')
        plt.title('Input image')
        #plt.show()
        '''
        plt.figure()
        #plt.subplot(132)
        cmap = plt.cm.jet
        plt.imshow(y_predprob, cmap = cmap)
        plt.axis('off')
        #plt.title('Probabilistic image')
        plt.savefig("/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/test_results/probabilistic_map/" +textx_file_name+ "", bbox_inches = 'tight', pad_inches = 0)
        plt.show()

        '''
        plt.subplot(133)
        plt.imshow(y_pred)#, cmap="Greys_r")
        plt.axis('off')
        plt.title('Binary image')
        plt.show()
        '''
        #cv2.imwrite(f"/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/test_results/mask/{textx_file_name}", y)
        #cv2.imwrite(f"/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/test_results/binary_map/{textx_file_name}", y_pred)
        #cv2.imwrite(f"/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/test_results/probabilistic_map/Probresults_{i}.png", y_predprob)
        
        '''
        h, w, _ = x.shape

        y_pred = unetModel.predict(np.expand_dims(x, axis=0))
        y_pred = y_pred[0] > 0.5

        white_line = np.ones((h, 10, 3)) * 255.0
        all_images = [
                x * 255.0, white_line,
                mask_parse(y), white_line,
                mask_parse(y_pred) * 255.0
        ]
        image = np.concatenate(all_images, axis=1)
        cv2.imwrite(f"/content/drive/MyDrive/Colab Notebooks/My work/billboard_unet_seg/test_results/org_gt_pred_map/Binaryresults_{i}.png", image)
        '''
        

Output hidden; open in https://colab.research.google.com to view.

In [23]:
from sklearn.utils.multiclass import type_of_target
print('y_pred', y_pred)
print(type_of_target(y_pred))
print('y', y)
y = y.astype(int)
print(type_of_target(y))


y_pred [[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
unknown
y [[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]

In [25]:
from sklearn import metrics



#fpr, tpr, thresholds = metrics.roc_curve(y, y_pred)#, pos_label=2)#y_true

fpr, tpr, _ = metrics.roc_curve(y, y_pred)
auc = metrics.roc_auc_score(y, y_pred)

ValueError: ignored